In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df_infos = pd.read_csv('../saved/infos.csv')
fDataReNorms = np.load('../saved/fDataReNorms.npy')

In [4]:
#df_infos

In [5]:
# from sklearn.preprocessing import normalize
# #fDataReNorms=[fDataRe-np.tile(fDataRe[:,0],(250,1)).transpose() for fDataRe in fDataRes]
# fDataReNorms=np.array([normalize(fDataRe) for fDataRe in fDataRes])
# np.save('../saved/fDataReNorms.npy', fDataReNorms)
# [plt.plot(fDataReNorm[6]) for fDataReNorm in fDataReNorms]

In [6]:
# fDataReNormAFs = fDataReNorms[df_infos['AF']]

# for fDataReNormAF in fDataReNormAFs[:10]:
#     plt.plot(fDataReNormAF[0])
#     plt.show()

In [7]:
from collections import Counter
from xgb.global_vars import headers, labels
print(Counter(np.sum(df_infos[labels], axis=1)))

Counter({1: 100528})


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [9]:
y = torch.FloatTensor(df_infos[labels].to_numpy()).to(device)

In [10]:
X = torch.FloatTensor(fDataReNorms).to(device)

In [11]:

class SimpleNet(nn.Module):
    def __init__(self, n_input, n_h1, n_h2, n_h3, verbose=False):
        super(SimpleNet, self).__init__()
        self.fc1        = nn.Linear(n_input, n_h1)
        self.fc2        = nn.Linear(n_h1, n_h2)
        self.fc3        = nn.Linear(120, n_h3)
        self.fc4        = nn.Linear(n_h3, 9)
        self.dropout = nn.Dropout(p=0.5)
        self.verbose=verbose

    def forward(self, x):
        if self.verbose:
            print('0: ', x.shape)
        x = self.fc1( x )     
        x = self.dropout(x)
        if self.verbose:
            print('1: ', x.shape)
        x = F.relu(   x )     
        x = self.fc2( x )    
        if self.verbose:
            print('2: ', x.shape)
        x = F.relu(   x )      
        if self.verbose:
            print('3: ', x.shape)
        x = x.view(x.shape[0], -1)
        x = self.fc3( x )  
        if self.verbose:
            print('4: ', x.shape)
        x = F.relu(   x )   
        x = self.fc4(x)
        if self.verbose:
            print('5: ', x.shape)
        return x

model = SimpleNet(250, 250, 10, 10)
model.forward(X.cpu())

tensor([[ 0.2487, -0.0068,  0.1357,  ...,  0.2155,  0.1215,  0.3133],
        [ 0.2495, -0.0049,  0.1391,  ...,  0.2178,  0.1217,  0.3125],
        [ 0.2515, -0.0049,  0.1423,  ...,  0.2213,  0.1201,  0.3112],
        ...,
        [ 0.2515, -0.0111,  0.1362,  ...,  0.2203,  0.1170,  0.3167],
        [ 0.2490, -0.0043,  0.1393,  ...,  0.2201,  0.1224,  0.3141],
        [ 0.2426, -0.0004,  0.1352,  ...,  0.2249,  0.1298,  0.3138]],
       grad_fn=<AddmmBackward>)

In [12]:
#%%
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
from evaluate_12ECG_score import compute_beta_score, compute_auc

# def evaluate_beta(output, y):
    
#     accuracy,f_measure,f_beta,g_beta = compute_beta_score(labels=y, 
#                        output=output, 
#                        beta=2, num_classes=1)
    
#     auroc, auprc = compute_auc(labels=y, 
#                                 probabilities=output,
#                                 num_classes=1)

#     return accuracy,f_measure,f_beta,g_beta, auroc, auprc


def confusion(prediction, truth):
    
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    https://gist.github.com/the-bass/cae9f3976866776dea17a5049013258d
    """

    confusion_vector = prediction / truth
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives

def binary_acc(y_preds, y_tests, beta=2):
    accs = []
    fmeasures = []
    fbetas = []
    gbetas = []
    aurocs = []
    auprcs = []
    for i in range(9):
        y_pred, y_test = y_preds[:,i], y_tests[:,i]
        
        # prob
        y_pred_prob = torch.sigmoid(y_pred)
        
        # auroc
        y_test_numpy = y_test.data.cpu().numpy()
        y_pred_prob_numpy = y_pred_prob.data.cpu().numpy()
        auroc = roc_auc_score(y_test_numpy, y_pred_prob_numpy)
        
        # auprc
        precision, recall, thresholds = precision_recall_curve(y_test_numpy, y_pred_prob_numpy)
        auprc = auc(recall, precision)

        # binary result
        y_pred_tag = torch.round(y_pred_prob)
        tp, fp, tn, fn = confusion(y_pred_tag, y_test)
        
        # acc, fmeasure, fbeta, gbeta
        acc = float(tp + tn) / float(tp + fp + fn + tn)
        fmeasure = float(2 * tp) / float(2 * tp + fp + fn)
        fbeta = float((1+beta**2)* tp) / float(((1+beta**2)*tp) + (fn*beta**2) + fp)
        gbeta = float(tp) / float(tp + fp + beta*fn)
        
        # old way to cal acc:
        #correct_results_sum = (y_pred_tag == y_test).sum().float()
        #acc = true_positives/y_test.shape[0]
        #acc = torch.round(acc * 100)

        accs.append(acc)#.data.cpu().numpy())
        fbetas.append(fbeta)#.data.cpu().numpy())
        fmeasures.append(fmeasure)
        gbetas.append(gbeta)
        aurocs.append(auroc)
        auprcs.append(auprc)
    return np.mean(accs), np.mean(fbetas), np.mean(fmeasures), np.mean(gbetas), np.mean(aurocs), np.mean(auprcs)

from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import KFold
import time

st = time.time()
patience = 200
kf = KFold(10)

saved_dir = '../saved/model/'
for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    model = SimpleNet(250, 250, 10, 10).to(device)
    learning_rate = 0.01
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    criterion = nn.BCEWithLogitsLoss()

    losses_train = []
    losses_test = []

    avg_losses_train = []
    avg_losses_test = []


    early_stopping = EarlyStopping(patience, verbose=False, 
                                  saved_dir=saved_dir, 
                                   save_name='SimpleNet'+str(i))
    epoch = 0
    auroc = 0
    auprc = 0
    accuracy = 0
    fmeasure = 0
    fbeta = 0
    gbeta = 0
    for epoch in range(10000):
        train_loss = 0
        validation_loss = 0

        optimizer.zero_grad()
        output_train = model(X[train_idx])
        output_test = model(X[test_idx])

        loss_train = criterion(output_train, y[train_idx])
        loss_test = criterion(output_test, y[test_idx])

        loss_train.backward()
        optimizer.step()

        losses_train.append(loss_train.item())
        losses_test.append(loss_test.item())

        avg_loss_train = np.average(losses_train)
        avg_loss_test = np.average(losses_test)

        avg_losses_train.append(avg_loss_train)
        avg_losses_test.append(avg_loss_test)

        if epoch % 200 == 0:
            acc, fmeasure, fbeta, gbeta, auroc, auprc = binary_acc(output_train, y[train_idx])
            acc2, fmeasure2, fbeta2, gbeta2, auroc2, auprc2 = binary_acc(output_test, y[test_idx])
            #accuracy_p = ' '.join(['{:.2f},'.format(acc)for acc in accuracy])
            #accuracy2_p = ' '.join(['{:.2f},'.format(acc)for acc in accuracy2])
            print('S{}/{} {:.2f} min|\nTrain Loss: {:.6f}, Acc: {:.2f}, F: {:.2f}, Fbeta: {:.2f}, gbeta: {:.2f}, auroc: {:.2f}, auprc: {:.2f} |\nValid Loss: {:.6f}, Acc: {:.2f}, F: {:.2f}, Fbeta: {:.2f}, gbeta: {:.2f}, auroc: {:.2f}, auprc: {:.2f}\n '.format(
                i, epoch, (time.time()-st)/60,
                avg_loss_train, acc, fmeasure, fbeta, gbeta, auroc, auprc, 
                avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, auroc2, auprc2))

        early_stopping(avg_loss_test, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break
            
    output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc,auprc,acc,fmeasure,fbeta,gbeta)
    print(output_string)     
    with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
        f.write(output_string)

    avg_losses_train = np.array(avg_losses_train)
    avg_losses_test = np.array(avg_losses_test)
    
    np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
    np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)
    

S0/0 0.01 min|
Train Loss: 0.746458, Acc: 0.31, F: 0.31, Fbeta: 0.17, gbeta: 0.10, auroc: 0.52, auprc: 0.12 |
Valid Loss: 0.745841, Acc: 0.31, F: 0.31, Fbeta: 0.17, gbeta: 0.10, auroc: 0.53, auprc: 0.12
 
S0/200 0.79 min|
Train Loss: 0.253087, Acc: 0.93, F: 0.35, Fbeta: 0.37, gbeta: 0.22, auroc: 0.88, auprc: 0.52 |
Valid Loss: 0.254342, Acc: 0.92, F: 0.34, Fbeta: 0.35, gbeta: 0.21, auroc: 0.87, auprc: 0.48
 
S0/400 1.56 min|
Train Loss: 0.219793, Acc: 0.93, F: 0.40, Fbeta: 0.42, gbeta: 0.27, auroc: 0.90, auprc: 0.57 |
Valid Loss: 0.228372, Acc: 0.92, F: 0.38, Fbeta: 0.39, gbeta: 0.24, auroc: 0.88, auprc: 0.52
 
S0/600 2.34 min|
Train Loss: 0.204346, Acc: 0.94, F: 0.42, Fbeta: 0.43, gbeta: 0.29, auroc: 0.91, auprc: 0.60 |
Valid Loss: 0.218410, Acc: 0.92, F: 0.38, Fbeta: 0.39, gbeta: 0.24, auroc: 0.88, auprc: 0.52
 
S0/800 3.11 min|
Train Loss: 0.194382, Acc: 0.94, F: 0.46, Fbeta: 0.49, gbeta: 0.31, auroc: 0.92, auprc: 0.64 |
Valid Loss: 0.212785, Acc: 0.93, F: 0.41, Fbeta: 0.43, gbeta: 

S2/2200 28.99 min|
Train Loss: 0.184518, Acc: 0.95, F: 0.59, Fbeta: 0.63, gbeta: 0.41, auroc: 0.94, auprc: 0.74 |
Valid Loss: 0.226217, Acc: 0.93, F: 0.50, Fbeta: 0.54, gbeta: 0.32, auroc: 0.89, auprc: 0.61
 
S2/2400 29.76 min|
Train Loss: 0.180434, Acc: 0.95, F: 0.61, Fbeta: 0.65, gbeta: 0.43, auroc: 0.95, auprc: 0.75 |
Valid Loss: 0.224404, Acc: 0.93, F: 0.51, Fbeta: 0.54, gbeta: 0.33, auroc: 0.89, auprc: 0.61
 
S2/2600 30.54 min|
Train Loss: 0.176783, Acc: 0.95, F: 0.62, Fbeta: 0.66, gbeta: 0.43, auroc: 0.95, auprc: 0.76 |
Valid Loss: 0.222855, Acc: 0.93, F: 0.52, Fbeta: 0.55, gbeta: 0.33, auroc: 0.89, auprc: 0.61
 
S2/2800 31.31 min|
Train Loss: 0.173511, Acc: 0.95, F: 0.63, Fbeta: 0.67, gbeta: 0.45, auroc: 0.95, auprc: 0.77 |
Valid Loss: 0.221558, Acc: 0.93, F: 0.53, Fbeta: 0.56, gbeta: 0.34, auroc: 0.90, auprc: 0.61
 
S2/3000 32.08 min|
Train Loss: 0.170293, Acc: 0.95, F: 0.66, Fbeta: 0.71, gbeta: 0.47, auroc: 0.96, auprc: 0.79 |
Valid Loss: 0.220235, Acc: 0.93, F: 0.55, Fbeta: 0

S3/2800 58.95 min|
Train Loss: 0.145031, Acc: 0.96, F: 0.68, Fbeta: 0.72, gbeta: 0.47, auroc: 0.96, auprc: 0.80 |
Valid Loss: 0.188137, Acc: 0.94, F: 0.55, Fbeta: 0.58, gbeta: 0.35, auroc: 0.90, auprc: 0.64
 
S3/3000 59.72 min|
Train Loss: 0.143525, Acc: 0.96, F: 0.68, Fbeta: 0.72, gbeta: 0.47, auroc: 0.96, auprc: 0.80 |
Valid Loss: 0.188063, Acc: 0.93, F: 0.55, Fbeta: 0.58, gbeta: 0.35, auroc: 0.90, auprc: 0.64
 
S3/3200 60.49 min|
Train Loss: 0.142166, Acc: 0.96, F: 0.69, Fbeta: 0.73, gbeta: 0.48, auroc: 0.96, auprc: 0.80 |
Valid Loss: 0.188033, Acc: 0.94, F: 0.54, Fbeta: 0.58, gbeta: 0.34, auroc: 0.90, auprc: 0.64
 
S3/3400 61.26 min|
Train Loss: 0.140935, Acc: 0.96, F: 0.69, Fbeta: 0.73, gbeta: 0.48, auroc: 0.96, auprc: 0.81 |
Valid Loss: 0.188029, Acc: 0.94, F: 0.55, Fbeta: 0.58, gbeta: 0.35, auroc: 0.90, auprc: 0.64
 
Early stopping
AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure
0.959|0.806|0.000|0.688|0.729|0.481
S4/0 61.72 min|
Train Loss: 0.696923, Acc: 0.53, F: 0.

S5/1800 88.33 min|
Train Loss: 0.170814, Acc: 0.95, F: 0.54, Fbeta: 0.57, gbeta: 0.37, auroc: 0.94, auprc: 0.69 |
Valid Loss: 0.209167, Acc: 0.93, F: 0.46, Fbeta: 0.49, gbeta: 0.28, auroc: 0.88, auprc: 0.57
 
S5/2000 89.10 min|
Train Loss: 0.167892, Acc: 0.95, F: 0.55, Fbeta: 0.59, gbeta: 0.38, auroc: 0.94, auprc: 0.70 |
Valid Loss: 0.208702, Acc: 0.93, F: 0.46, Fbeta: 0.50, gbeta: 0.29, auroc: 0.88, auprc: 0.57
 
S5/2200 89.87 min|
Train Loss: 0.165300, Acc: 0.95, F: 0.55, Fbeta: 0.59, gbeta: 0.39, auroc: 0.94, auprc: 0.71 |
Valid Loss: 0.208383, Acc: 0.93, F: 0.46, Fbeta: 0.49, gbeta: 0.28, auroc: 0.88, auprc: 0.57
 
S5/2400 90.63 min|
Train Loss: 0.162999, Acc: 0.95, F: 0.56, Fbeta: 0.60, gbeta: 0.39, auroc: 0.94, auprc: 0.72 |
Valid Loss: 0.208102, Acc: 0.93, F: 0.47, Fbeta: 0.50, gbeta: 0.29, auroc: 0.88, auprc: 0.58
 
S5/2600 91.40 min|
Train Loss: 0.160940, Acc: 0.95, F: 0.57, Fbeta: 0.61, gbeta: 0.40, auroc: 0.94, auprc: 0.72 |
Valid Loss: 0.207913, Acc: 0.93, F: 0.47, Fbeta: 0

S6/2200 118.93 min|
Train Loss: 0.158247, Acc: 0.95, F: 0.61, Fbeta: 0.64, gbeta: 0.43, auroc: 0.95, auprc: 0.75 |
Valid Loss: 0.209138, Acc: 0.93, F: 0.48, Fbeta: 0.51, gbeta: 0.30, auroc: 0.89, auprc: 0.57
 
Early stopping
AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure
0.947|0.751|0.000|0.607|0.643|0.430
S7/0 119.41 min|
Train Loss: 0.700673, Acc: 0.41, F: 0.15, Fbeta: 0.07, gbeta: 0.04, auroc: 0.47, auprc: 0.10 |
Valid Loss: 0.701918, Acc: 0.40, F: 0.13, Fbeta: 0.06, gbeta: 0.04, auroc: 0.47, auprc: 0.11
 
S7/200 120.19 min|
Train Loss: 0.260374, Acc: 0.92, F: 0.39, Fbeta: 0.42, gbeta: 0.24, auroc: 0.87, auprc: 0.52 |
Valid Loss: 0.262897, Acc: 0.92, F: 0.35, Fbeta: 0.39, gbeta: 0.21, auroc: 0.84, auprc: 0.46
 
S7/400 120.97 min|
Train Loss: 0.223658, Acc: 0.93, F: 0.46, Fbeta: 0.48, gbeta: 0.30, auroc: 0.90, auprc: 0.59 |
Valid Loss: 0.235684, Acc: 0.93, F: 0.41, Fbeta: 0.44, gbeta: 0.25, auroc: 0.86, auprc: 0.53
 
S7/600 121.75 min|
Train Loss: 0.206243, Acc: 0.94, F: 

S8/2800 148.07 min|
Train Loss: 0.146096, Acc: 0.96, F: 0.69, Fbeta: 0.73, gbeta: 0.48, auroc: 0.96, auprc: 0.80 |
Valid Loss: 0.188222, Acc: 0.94, F: 0.57, Fbeta: 0.61, gbeta: 0.37, auroc: 0.92, auprc: 0.67
 
S8/3000 148.84 min|
Train Loss: 0.144397, Acc: 0.96, F: 0.70, Fbeta: 0.74, gbeta: 0.49, auroc: 0.96, auprc: 0.81 |
Valid Loss: 0.188017, Acc: 0.94, F: 0.57, Fbeta: 0.60, gbeta: 0.36, auroc: 0.91, auprc: 0.66
 
S8/3200 149.61 min|
Train Loss: 0.142865, Acc: 0.96, F: 0.69, Fbeta: 0.73, gbeta: 0.49, auroc: 0.96, auprc: 0.81 |
Valid Loss: 0.187858, Acc: 0.94, F: 0.56, Fbeta: 0.60, gbeta: 0.36, auroc: 0.91, auprc: 0.66
 
S8/3400 150.38 min|
Train Loss: 0.141471, Acc: 0.96, F: 0.69, Fbeta: 0.73, gbeta: 0.48, auroc: 0.96, auprc: 0.81 |
Valid Loss: 0.187737, Acc: 0.94, F: 0.56, Fbeta: 0.59, gbeta: 0.36, auroc: 0.91, auprc: 0.66
 
S8/3600 151.15 min|
Train Loss: 0.140095, Acc: 0.96, F: 0.70, Fbeta: 0.74, gbeta: 0.50, auroc: 0.96, auprc: 0.82 |
Valid Loss: 0.187612, Acc: 0.94, F: 0.57, Fbe